# Build a system to auto-catagorize 16s rRNAs, ITS and blastn search

## some constants

### locations

 content | location 
 -------: | :--------
 blast+ folder | `/home/p/blast/bin/`
 blastn | `/home/p/blast/bin/blastn`
 ascp | `/home/p/ascp/connect/bin/ascp`
 ~ | `/mnt/d/linux`

### import required modules

In [74]:
import pandas as pd
import os
import sys
import shutil
import re
import pickle

In [67]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import time
from Bio import SeqIO
from io import StringIO
from collections import Counter

## download data

### download taxonamy, gss, wgs, est

run with linux
```
/home/p/ascp/connect/bin/ascp -T -k 1 -i /home/p/ascp/connect/etc/asperaweb_id_dsa.openssh anonftp@ftp.ncbi.nlm.nih.gov:/pub/taxonomy/accession2taxid/nucl_gss.accession2taxid.gz  /mnt/d/linux/W/NCBI/taxonamy/

/home/p/ascp/connect/bin/ascp -T -k 1 -i /home/p/ascp/connect/etc/asperaweb_id_dsa.openssh anonftp@ftp.ncbi.nlm.nih.gov:/pub/taxonomy/accession2taxid/nucl_wgs.accession2taxid.gz  /mnt/d/linux/W/NCBI/taxonamy/

/home/p/ascp/connect/bin/ascp -T -k 1 -i /home/p/ascp/connect/etc/asperaweb_id_dsa.openssh anonftp@ftp.ncbi.nlm.nih.gov:/pub/taxonomy/accession2taxid/nucl_est.accession2taxid.gz  /mnt/d/linux/W/NCBI/taxonamy/

/home/p/ascp/connect/bin/ascp -T -k 1 -i /home/p/ascp/connect/etc/asperaweb_id_dsa.openssh anonftp@ftp.ncbi.nlm.nih.gov:/pub/taxonomy/accession2taxid/nucl_gb.accession2taxid.gz  /mnt/d/linux/W/NCBI/taxonamy/


/home/p/ascp/connect/bin/ascp -T -k 1 -i /home/p/ascp/connect/etc/asperaweb_id_dsa.openssh anonftp@ftp.ncbi.nlm.nih.gov:/pub/taxonomy/taxdump.tar.gz  /mnt/d/linux/W/NCBI/taxonamy/
```

### download nt
run with linux
```

/home/p/ascp/connect/bin/ascp -T -k 1 -i /home/p/ascp/connect/etc/asperaweb_id_dsa.openssh anonftp@ftp.ncbi.nlm.nih.gov:/blast/db/FASTA//nt.gz  /mnt/d/linux/W/NCBI/nt20180810/

```

### unzip the data


```
cd /mnt/d/linux/W/NCBI/taxonamy20180810
gzip -d *
tar xf taxdump.tar
rm taxdump.tar

```

## taxonomy

Update the data to 20180810.  
Use new method to include the whole lineage of different species. Use new method to retrieve taxonomy.

### taxonomy extract scientific name


In [29]:
file_tax_name = '/mnt/d/linux/W/NCBI/taxonamy20180810/names.dmp'
import pandas as pd
df_name = pd.read_csv(file_tax_name,sep = "\t", header = None,dtype = str)

df_name = df_name.loc[:,[0,2,6]]
df_name_keep = df_name[df_name[6] == "scientific name"]

df_name_keep = df_name_keep.loc[:,[0,2]]
df_name_keep.columns = ["id","name"]
df_name_keep.to_csv("/mnt/d/linux/W/NCBI/taxonamy20180810/speciesName", sep = "\t", float_format = "%d", index = False)

### taxonomy, clear nodes.dmp, only keet 'tax_id, parent_id,rank'


In [ ]:
file_tax_nodes = '/mnt/d/linux/W/NCBI/taxonamy20180810/nodes.dmp'
import pandas as pd
df_node = pd.read_csv(file_tax_nodes, sep = "\t",header = None,dtype = str)
df_node = df_node.loc[:,[0,2,4]]
df_node.columns = ["From","To","rank"]
df_node.head()
df_node.to_csv("/mnt/d/linux/W/NCBI/taxonamy20180810/nodesLink", sep = "\t", float_format = "%d", index = False)

### taxonomy, creat speciesLineage, all tax_ids, and find all their lineage.
total tax_ids: 1832355. 

In [45]:
import pandas as pd
file_tax_nodesLink = "/mnt/d/linux/W/NCBI/taxonamy20180810/nodesLink"
df_node = pd.read_csv(file_tax_nodesLink, sep = "\t",dtype = str)
print(df_node.shape)
ls_all_tax_ids = list(df_node['From'])
print('total tax_ids:', len(ls_all_tax_ids))
dc_node = dict(zip(list(df_node.loc[:,"From"]), list(df_node.loc[:,"To"])))
dc_rank = dict(zip(list(df_node.loc[:,"From"]), list(df_node.loc[:,"rank"])))

ls_all_tax_ids_lineage = []
for _id in ls_all_tax_ids:
    _From = _id
    _To = dc_node[_From]
    _id_lineage = []
    while _To != _From:
        _id_lineage.append(_From)
        _From = _To
        _To = dc_node[_From]
    _id_lineage.append(_To)
    ls_all_tax_ids_lineage.append(_id_lineage)
#    break
print(len(ls_all_tax_ids_lineage))

fout = open('/mnt/d/linux/W/NCBI/taxonamy20180810/speciesLineage','w')
for _e in ls_all_tax_ids_lineage:
    fout.write('\t'.join(_e)+'\n')
fout.close()

(1832355, 3)
total tax_ids: 1832355
1832355


### taxonomy, create speciesLineageName.
For all lines in speciesLineage, if there is any element of them have the level of "species", keep it. Then the speciesLineageName. Columns tax_id, name, species_id, species, genus, family, order, class, phylum, kingdom.  
For kingdom, keep those IDs {"2","2157","12884","10239","4751","33090","33208"}  

In [46]:
## generate speciesLineageName
##  species, genus, family, order, class, phylum, kingdom.
df_name = pd.read_csv('/mnt/d/linux/W/NCBI/taxonamy20180810/speciesName',sep='\t',dtype=str)
dc_name = dict(zip(df_name['id'],df_name['name']))
kingdom_ids =  {"2","2157","12884","10239","4751","33090","33208"}
rank_ids = {'genus','family','order','class','phylum'}
ls_species_tax_ids_lineage = []
for _l in ls_all_tax_ids_lineage:
    for _id in _l:
        if dc_rank[_id] == 'species':
            ls_species_tax_ids_lineage.append(_l)
            break
print(len(ls_species_tax_ids_lineage))
ls_species_tax_ids_dc = []
for _l in ls_species_tax_ids_lineage:
    _dc = {}
    _dc['tax_id'] = _l[0]
    if _id in dc_name:
        _dc['name'] = dc_name[_l[0]]
    for _id in _l:
        if _id in kingdom_ids:
            if _id in dc_name:
                _dc['kingdom'] = dc_name[_id]
        if dc_rank[_id] == 'species':
            _dc['species_id'] = _id
            if _id in dc_name:
                _dc['species'] = dc_name[_id]
        if dc_rank[_id] in rank_ids:
            if _id in dc_name:
                _dc[dc_rank[_id]] = dc_name[_id]
    ls_species_tax_ids_dc.append(_dc)

df_lineage = pd.DataFrame(ls_species_tax_ids_dc, dtype=str)
df_lineage = df_lineage.loc[:,['tax_id','species_id',  'name','species','genus','family','order','class','phylum', 'kingdom']]
df_lineage.to_csv('/mnt/d/linux/W/NCBI/taxonamy20180810/speciesLineageName',sep='\t', index=None)

1713575


### store the speciesLineageName dataframe in binary format to speed up future usage

In [75]:
df_goodLineage = pd.read_csv("/mnt/d/linux/W/NCBI/taxonamy20180810/speciesLineageName", sep = "\t", dtype = str)
df_goodLineage.index = df_goodLineage["tax_id"]
df_goodLineage = df_goodLineage.drop(labels=["tax_id"], axis = 1)
pickle.dump(df_goodLineage,open('/mnt/d/linux/W/NCBI/taxonamy20180810/speciesLineageName.memory','wb'))

In [77]:
df_goodLineage.head()

,species_id,name,species,genus,family,order,class,phylum,kingdom
tax_id,,,,,,,,,
7,7,Azorhizobium caulinodans,Azorhizobium caulinodans,Azorhizobium,Xanthobacteraceae,Rhizobiales,Alphaproteobacteria,Proteobacteria,Bacteria
9,9,Buchnera aphidicola,Buchnera aphidicola,Buchnera,Erwiniaceae,Enterobacterales,Gammaproteobacteria,Proteobacteria,Bacteria
11,11,Cellulomonas gilvus,Cellulomonas gilvus,Cellulomonas,Cellulomonadaceae,Micrococcales,Actinobacteria,Actinobacteria,Bacteria
14,14,Dictyoglomus thermophilum,Dictyoglomus thermophilum,Dictyoglomus,Dictyoglomaceae,Dictyoglomales,Dictyoglomia,Dictyoglomi,Bacteria
17,17,Methylophilus methylotrophus,Methylophilus methylotrophus,Methylophilus,Methylophilaceae,Nitrosomonadales,Betaproteobacteria,Proteobacteria,Bacteria


## Ezbio
update the data to 20180810.  
From this webpage `https://www.ezbiocloud.net/16SrRNA_list?tn=Root` to get the accession numbers of all 64,660 RNA sequences.  
The table is saved as `/mnt/d/linux/W/NCBI/Ezbio20180810/Ezbio20180810AccTable64660_16s_rRNA.txt`.

### Get 16s sequences

change working directory to Ezbio

In [4]:
os.chdir('/mnt/d/linux/W/NCBI/Ezbio20180810/')

read the Ezbio Acc Table file

In [5]:
df_EzTab = pd.read_csv('Ezbio20180810AccTable64660_16s_rRNA.txt', sep='\t',dtype=str)
df_EzTab.head()

,Accession,Taxon name,Strain name,Taxonomy
0,A61579,A61579_s,DSM 9460,Bacteria;Thermotogae;Thermotogae_c;Thermotogal...
1,AAAP01003867,Magnetospirillum magnetotacticum,MS-1(T),Bacteria;Proteobacteria;Alphaproteobacteria;Rh...
2,AABF01000026,Fusobacterium nucleatum subsp. vincentii,ATCC 49256(T),Bacteria;Fusobacteria;Fusobacteria_c;Fusobacte...
3,AABW01000001,Rickettsia sibirica,246(T),Bacteria;Proteobacteria;Alphaproteobacteria;Ri...
4,AACK01000032,Actinobacillus pleuropneumoniae,Shope 4074(T),Bacteria;Proteobacteria;Gammaproteobacteria;Pa...


get list of nucleotide sequences and download them from ncbi.  
Nucleotide: 1 letter + 5 numerals OR 2 letters + 6 numerals

In [7]:
pattern_acc_nuc = re.compile('^\w{1,2}\d*$',re.I)
ls_acc_nuc = [e for e in df_EzTab['Accession'] if re.match(pattern_acc_nuc,e)]
print(ls_acc_nuc[:5])
open('Ezbio20180810Acc_16s_rRNA_nucleotides.txt','w').write('\n'.join(ls_acc_nuc))

['A61579', 'AB000481', 'AB000700', 'AB001519', 'AB001520']


518752

In [27]:
print('Accession num:', len(df_EzTab['Accession']), "Accession num unique", len(set(df_EzTab['Accession'])))

Accession num: 64660 Accession num unique 54660


In [38]:
st_acc_Ez = set(df_EzTab['Accession'])

In [39]:
pattern_acc_nuc = re.compile('^\w{1,2}\d*$',re.I)
ls_acc_nuc = [e for e in st_acc_Ez if re.match(pattern_acc_nuc,e)]
st_acc_nuc = set(ls_acc_nuc)
ls_acc_notnuc = [e for e in st_acc_Ez if e not in st_acc_nuc]
pattern_acc_WGS = re.compile('^\w{4}\d*$',re.I)
ls_acc_WGS = [e for e in ls_acc_notnuc if re.match(pattern_acc_WGS,e)]
st_acc_WGS = set(ls_acc_WGS)
ls_acc_notWGS = [e for e in ls_acc_notnuc if e not in st_acc_WGS]
print('nucleotide',len(ls_acc_nuc),'WGS',len(ls_acc_WGS),'other',len(ls_acc_notWGS))

nucleotide 47914 WGS 6112 other 634


Remove the 634 sequences. Most of the are JGI:00000000 sequences, cannot find the seq from JGI.  
For nucleotide sequences, and WGS sequences, try to get them from SILVA first. 

In [40]:
from Bio import SeqIO
file_silva_fasta = '/mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA.fasta'
ls_fasta_silva = list(SeqIO.parse(open(file_silva_fasta),'fasta'))
dc_silva_fasta = {}
for _s in ls_fasta_silva:
    _acc = _s.id.split('.')[0]
    if _s.id.split('.')[0] not in dc_silva_fasta:
        dc_silva_fasta[_acc] = []
    dc_silva_fasta[_acc].append(_s)

ls_fasta_Ez_nuc = [dc_silva_fasta[e] for e in ls_acc_nuc if e in dc_silva_fasta]
ls_fasta_Ez_WGS = [dc_silva_fasta[e] for e in ls_acc_WGS if e in dc_silva_fasta]
ls_fasta_Ez_nuc_seqs = [i for j in ls_fasta_Ez_nuc for i in j]
ls_fasta_Ez_WGS_seqs = [i for j in ls_fasta_Ez_WGS for i in j]
ls_acc_missingSILVA = [e for e in ls_acc_nuc if e not in dc_silva_fasta] + [e for e in ls_acc_WGS if e not in dc_silva_fasta]
print('Ez_nuc in SILVA', len(ls_fasta_Ez_nuc,), 'seqs', len(ls_fasta_Ez_nuc_seqs))
print('Ez_WGS in SILVA', len(ls_fasta_Ez_WGS,), 'seqs', len(ls_fasta_Ez_WGS_seqs))

Ez_nuc in SILVA 45904 seqs 50306
Ez_WGS in SILVA 3260 seqs 3786


In [41]:
st_acc_missingSILVA = set(ls_acc_missingSILVA)
print('missing ids in SILVA', len(ls_acc_missingSILVA),"unique count:",len(st_acc_missingSILVA))

missing ids in SILVA 4862 unique count: 4862


define the function to download seq from Ezbio

In [19]:
def accEzbioWebRequest(acc):
    '''
    given a acc number, return a fasta seqrecord of Biopython
    '''
    n = 1
    while n<100:
        try:
            html = urlopen('https://www.ezbiocloud.net/16SrRNA?ac='+acc)
            
            txt = html.read().decode()
            seqs = re.findall(re.compile('data-clipboard-text=">{acc}.*?"'.format(acc=acc)),txt)
            if len(seqs) != 1:
                print(acc,'not exist')
                return None
            seq = seqs[0]
            seq = seq.replace('data-clipboard-text="','')
            seq = seq.replace('"','')
            seq = seq.replace('&#13;&#10;','\n')
            return SeqIO.read(StringIO(seq),'fasta')
        except:
            time.sleep(1)
            n += 1
            print('try the', n, 'time for ', acc)
    print('tried 100 times, no good for ', acc)
    return None

download sequences from Ezbio

In [21]:
dc_fasta_missingSILVA = {}

In [22]:
for _e in st_acc_missingSILVA:
    if _e not in dc_fasta_missingSILVA:
        dc_fasta_missingSILVA[_e] = accEzbioWebRequest(_e)
    _n = len(dc_fasta_missingSILVA)
    if _n % 100 == 0:
        print(_n,'finished')


100 finished
200 finished
300 finished
400 finished
500 finished
600 finished
700 finished
800 finished
900 finished
1000 finished
1100 finished
1200 finished
1300 finished
1400 finished
1500 finished
1600 finished
1700 finished
1800 finished
1900 finished
2000 finished
2100 finished
2200 finished
2300 finished
2400 finished
2500 finished
2600 finished
2700 finished
2800 finished
2900 finished
3000 finished
3100 finished
3200 finished
3300 finished
3400 finished
3500 finished
3600 finished
3700 finished
3800 finished
3900 finished
4000 finished
4100 finished
4200 finished
4300 finished
4400 finished
4500 finished
4600 finished
4700 finished
4800 finished


save sequences

In [42]:
ls_fasta_ez = list(dc_fasta_missingSILVA.values()) + ls_fasta_Ez_WGS_seqs + ls_fasta_Ez_nuc_seqs
print(len(ls_fasta_ez))

58954


In [43]:
fout = open('/mnt/d/linux/W/NCBI/Ezbio20180810/Ezbio20180810.fasta','w')
for _e in ls_fasta_ez:
    fout.write('>'+_e.id+'\n'+str(_e.seq)+'\n')
fout.close()

### get taxa for Ezbio

In [68]:
ls_fasta_ez = list(SeqIO.parse(open('/mnt/d/linux/W/NCBI/Ezbio20180810/Ezbio20180810.fasta'),'fasta'))
print(len(ls_fasta_ez))

58954


In [71]:
accs = set([e.id.split('.')[0] for e in ls_fasta_ez])
print(len(accs))

54026


In [72]:
dc_acc2tax = dict.fromkeys(accs)

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy20180810/nucl_gb.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
n=0
for l in fo:
    es = l.split()
    if es[0] in dc_acc2tax:
        dc_acc2tax[es[0]] = es[2]
        n+=1
fo.close()
print(len(dc_acc2tax),n)

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy20180810/nucl_gss.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
n = 0
for l in fo:
    es = l.split()
    if es[0] in dc_acc2tax:
        n+=1
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax),n)

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy20180810/nucl_wgs.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
n=0
for l in fo:
    es = l.split()
    if es[0] in dc_acc2tax:
        dc_acc2tax[es[0]] = es[2]
        n+=1
fo.close()
print(len(dc_acc2tax),n)

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy20180810/nucl_est.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
n=0
for l in fo:
    es = l.split()
    if es[0] in dc_acc2tax:
        dc_acc2tax[es[0]] = es[2]
        n+=1
fo.close()
print(len(dc_acc2tax),n)


fout = open("/mnt/d/linux/W/NCBI/Ezbio20180810/Ezbio20180810.taxa","w")
fout2 = open("/mnt/d/linux/W/NCBI/Ezbio20180810/Ezbio20180810.taxaRare","w")
for _k,_v in dc_acc2tax.items():
    if _v:
        fout.write(_k + "\t" + _v +"\n")
    else:
        fout2.write(_k + "\n")
fout.close()
fout2.close()

54026 47430
54026 0
54026 3797
54026 0


2799 accs do not have taxID. Find those sequences, and replace accession number with newest version in NCBI.
```
/home/p/blast/bin/blastn -db /mnt/d/linux/W/NCBI/nt20180810/nt_micro -query /mnt/d/linux/W/NCBI/Ezbio20180810/Ezbio20180810.taxaRare.fasta  -out /mnt/d/linux/W/NCBI/Ezbio20180810/Ezbio20180810.taxaRare.fasta.nt.tab -num_threads 24 -max_target_seqs 10 -word_size 100 -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovhsp staxids'

/home/p/blast/bin/blastn -db /mnt/d/linux/W/NCBI/SILVA/SILVA16sDNA -query /mnt/d/linux/W/NCBI/Ezbio20180810/Ezbio20180810.taxaRare.fasta  -out /mnt/d/linux/W/NCBI/Ezbio20180810/Ezbio20180810.taxaRare.fasta.SILVA.tab -num_threads 24 -max_target_seqs 10 -word_size 100 -outfmt '6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore qcovhsp staxids'
```
It's hard to find sequences 100% identical to them.

In [73]:
st_ez_acc_noTax = set(open("/mnt/d/linux/W/NCBI/Ezbio20180810/Ezbio20180810.taxaRare").read().split())
ls_fasta_ez_noTax = [e for e in ls_fasta_ez if e.id.split('.')[0] in st_ez_acc_noTax]
print(len(ls_fasta_ez_noTax))
fout = open("/mnt/d/linux/W/NCBI/Ezbio20180810/Ezbio20180810.taxaRare.fasta",'w')
for e in ls_fasta_ez_noTax:
    fout.write('>'+e.id+'\n'+str(e.seq)+'\n')
fout.close()

2799


### make blast DB

run with linux

```
cd /mnt/d/linux/W/NCBI/Ezbio20180810
/home/p/blast/bin/dustmasker -in Ezbio20180810.fasta -infmt fasta -parse_seqids -outfmt maskinfo_asn1_bin -out Ezbio20180810.asnb

/home/p/blast/bin/makeblastdb -in Ezbio20180810.fasta -input_type fasta -dbtype nucl -mask_data Ezbio20180810.asnb -out Ezbio20180810 -title "Ezbio20180810" -parse_seqids -taxid_map /mnt/d/linux/W/NCBI/Ezbio20180810/Ezbio20180810.taxa

```

## nt

### extract taxid from nt

In [30]:
f_nt = "/mnt/d/linux/W/NCBI/nt20180810/nt"
from Bio import SeqIO
seqs = SeqIO.parse(open(f_nt), "fasta")
accs = []
for seq in seqs:
    accs.append(seq.id.split(".")[0])
print(len(accs))

# get taxid
#accs = set(accs)
#creat a dictionary with accession number as key, tax id as value

49326191


KeyboardInterrupt: 

In [ ]:
dc_acc2tax = dict.fromkeys(accs)

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy20180810/nucl_gb.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
n=0
for l in fo:
    es = l.split()
    if es[0] in dc_acc2tax:
        dc_acc2tax[es[0]] = es[2]
        n+=1
fo.close()
print(len(dc_acc2tax),n)

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy20180810/nucl_gss.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
n = 0
for l in fo:
    es = l.split()
    if es[0] in dc_acc2tax:
        n+=1
        dc_acc2tax[es[0]] = es[2]
fo.close()
print(len(dc_acc2tax),n)

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy20180810/nucl_wgs.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
n=0
for l in fo:
    es = l.split()
    if es[0] in dc_acc2tax:
        dc_acc2tax[es[0]] = es[2]
        n+=1
fo.close()
print(len(dc_acc2tax),n)

f_acc2tax = "/mnt/d/linux/W/NCBI//taxonamy20180810/nucl_est.accession2taxid"
fo = open(f_acc2tax)
fo.readline()
n=0
for l in fo:
    es = l.split()
    if es[0] in dc_acc2tax:
        dc_acc2tax[es[0]] = es[2]
        n+=1
fo.close()
print(len(dc_acc2tax),n)


fout = open("/mnt/d/linux/W/NCBI/nt20180810/nt.taxa","w")
fout2 = open("/mnt/d/linux/W/NCBI/nt20180810/nt.taxaRare","w")
for _k,_v in dc_acc2tax.items():
    if _v:
        fout.write(_k + "\t" + _v +"\n")
    else:
        fout2.write(_k + "\n")
fout.close()
fout2.close()

In [32]:
open('/mnt/d/linux/W/NCBI/nt20180810/nt.acc','w').write('\n'.join(accs))

534660009

### find taxa for nt.taxaRare

Those IDs might be too new and not included in the database yet. Use python code to get there taxonomy

In [34]:
## extract taxonomy from nt_taxRare
def accNCBIgetTax(accs):
    '''
    given a list of acc numbers, return list of acc and taxid
    '''
    n = 1
    acc_join = ','.join(accs)
    while n<100:
        try:
            html = urlopen('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=nucleotide&id='+acc_join)
            
            txt = html.read().decode()
            txts = txt.split('</DocSum>')
            taxids = []
            for _n,acc in enumerate(accs):
                _m = re.search('<Item Name="TaxId" Type="Integer">(\d*)</Item>',txts[_n])
                if _m:
                    tax_id = _m.group(1)
                taxids.append([acc, tax_id])
            return taxids
        except:
            time.sleep(1)
            n += 1
            print('try the', n, 'time')
    print('tried 100 times, no good')
    return None



In [ ]:
ls_nt_taxaRare = open('/mnt/d/linux/W/NCBI/nt20180810/nt.taxaRare').read().split()
ls_nt_taxaResult = []
for _i in range(0,len(ls_nt_taxaRare),300):
    print(_i,end=' ')
    ls_nt_taxaResult += accNCBIgetTax(ls_nt_taxaRare[_i:_i+300])
ls_nt_taxaResult_keep = [e for e in ls_nt_taxaResult if e[1]!='0']
fout = open('/mnt/d/linux/W/NCBI/nt20180810/nt.taxaRare.taxa','w')
for _e in ls_nt_taxaResult_keep:
    fout.write('\t'.join(_e)+'\n')
fout.close()

combine the taxa file

In [ ]:
os.system('cp /mnt/d/linux/W/NCBI/nt20180810/nt.taxa /mnt/d/linux/W/NCBI/nt20180810/nt.taxa.db')
os.system('cat /mnt/d/linux/W/NCBI/nt20180810/nt.taxaRare.taxa >>/mnt/d/linux/W/NCBI/nt20180810/nt.taxa')

### some taxids of nt accesion numbers are not in use any more. Change them

In [ ]:
dc_acc2taxid = {e.split()[0]:e.split()[1] for e in open("/mnt/d/linux/W/NCBI/nt20180810/nt.taxa")}

st_taxid_not_exist = set([e for e in dc_acc2taxid.values() if e not in dc_nt_taxid2group])
html = urlopen('https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=taxonomy&id='+','.join(st_taxid_not_exist))
txt = html.read().decode()
txts = txt.split('</DocSum>')
dc_taxid_correct = {}
for _n in range(len(txts)-1):
    _m = re.search('<Item Name="TaxId" Type="Integer">(\d*)</Item>',txts[_n])
    _mn = re.search('Item Name="AkaTaxId" Type="Integer">(\d*)</Item>',txts[_n])
    if _m and _mn:
        _k = _m.group(1)
        _v = _mn.group(1)
        if _v != '0':## new taxid is not available
            dc_taxid_correct[_k]=_v
fout = open('/mnt/d/linux/W/NCBI/taxonamy20180810/speciesChangedID','w')
for _k,_v in dc_taxid_correct.items():
    fout.write(_k+'\t'+_v+'\n')
fout.close()

n = 0
for _k,_v in dc_acc2taxid.items():
    if _v in dc_taxid_correct:
        dc_acc2taxid[_k] = dc_taxid_correct[_v]
        n +=1
print(n,'sequences with taxID corrected')
fout= open('/mnt/d/linux/W/NCBI/nt20180810/nt.taxa.corrected','w')
for _k,_v in dc_acc2taxid.items():
    fout.write(_k+'\t'+_v+'\n')
fout.close()

### 8 kingdoms/subkingdoms, split nt sequences to 8 groups based on kingdom.
{"2","2157","12884","10239","4751","33090","33208"}  
kingdom:
* Life
    * Archae 2157
    * Bacteria 2
    * Viroids 12884
    * Viruses 10239
    * Eukaryota 2759
        * Fungi 4751
        * viridiplantae 33090
        * metazoa 33208

accession to groups

In [ ]:
dc_nt_groups ={"2":"Bacteria", "2157":"Archae", "12884":"Viroids", "10239":"Viruses", \
               "4751":"Fungi", "33090":"viridiplantae" ,"33208":"metazoa", "":"Other"}
dc_nt_group_taxids = {e:[] for e in dc_nt_groups}
file_tax_speciesLineage = '/mnt/d/linux/W/NCBI/taxonamy20180810/speciesLineage'
for _l in open(file_tax_speciesLineage):
    _es = _l.split()
    _found = False
    for _e in _es:
        if _e in dc_nt_group_taxids:
            dc_nt_group_taxids[_e].append(_es[0])
            _found = True
            break
    if not _found:
        dc_nt_group_taxids[""].append(_es[0])

dc_nt_taxid2group = {} ## taxid to group, dictionary
for _k,_v in dc_nt_group_taxids.items():
    for _id in _v:
        dc_nt_taxid2group[_id] = _k

split the nt file

In [ ]:
dc_nt_groups_write = {k:open('/mnt/d/linux/W/NCBI/nt20180810/nt_'+v,'w') for k,v in dc_nt_groups.items()}

for _seq in SeqIO.parse(open('/mnt/d/linux/W/NCBI/nt20180810/nt'),'fasta'):
    _acc = _seq.id.split('.')[0]
    if _acc in dc_acc2taxid:
        _taxid = dc_acc2taxid[_acc]
        if _taxid in dc_nt_taxid2group:
            _kingdom = dc_nt_taxid2group[_taxid]
            _fout = dc_nt_groups_write[_kingdom]
            _fout.write('>'+_seq.description+'\n'+str(_seq.seq)+'\n')
        else:
            print(_taxid, "not in groups")
            dc_nt_groups_write[''].write('>'+_seq.description+'\n'+str(_seq.seq)+'\n')
    else:
        dc_nt_groups_write[''].write('>'+_seq.description+'\n'+str(_seq.seq)+'\n')

[e.close() for e in dc_nt_groups_write.values()]

### create nt_micro
by removing metazoa and viridiplantae

In [35]:
os.system("cat /mnt/d/linux/W/NCBI/nt20180810/nt_Archae >>/mnt/d/linux/W/NCBI/nt20180810/nt_micro")
os.system("cat /mnt/d/linux/W/NCBI/nt20180810/nt_Bacteria >>/mnt/d/linux/W/NCBI/nt20180810/nt_micro")
os.system("cat /mnt/d/linux/W/NCBI/nt20180810/nt_Fungi >>/mnt/d/linux/W/NCBI/nt20180810/nt_micro")
os.system("cat /mnt/d/linux/W/NCBI/nt20180810/nt_Other >>/mnt/d/linux/W/NCBI/nt20180810/nt_micro")
os.system("cat /mnt/d/linux/W/NCBI/nt20180810/nt_Viroids >>/mnt/d/linux/W/NCBI/nt20180810/nt_micro")
os.system("cat /mnt/d/linux/W/NCBI/nt20180810/nt_Viruses >>/mnt/d/linux/W/NCBI/nt20180810/nt_micro")

0

build database for nt_micro

run with linux
```
cd /mnt/d/linux/W/NCBI/nt20180810
/home/p/blast/bin/dustmasker -in nt_micro -infmt fasta -parse_seqids -outfmt maskinfo_asn1_bin -out nt_micro.asnb

/home/p/blast/bin/makeblastdb -in nt_micro -input_type fasta -dbtype nucl -mask_data nt_micro.asnb -out nt_micro -title "nt_micro" -parse_seqids -taxid_map nt.taxa.corrected
```

## MoonBio Human Bacteria

### Check if all taxids are "species" name
Totally 2690 species

In [78]:
file = "/mnt/d/linux/W/NCBI/MoonRef20180810/Human_bacterial_species_2018.txt"
df_species = pd.read_csv(file, sep='\t',dtype=str, index_col=0)
taxids = list(df_species.taxid)
taxids_s = set(taxids)
print(len(taxids),len(taxids_s))

2687 2687


In [79]:
df_species.head()

,Name_ori,taxid,sci_name,name_use
1,Abiotrophia defectiva,46125,Abiotrophia defectiva,Abiotrophia defectiva
2,Abiotrophia para-adiacens,191553,Abiotrophia para-adiacens,Abiotrophia para-adiacens
3,Acetobacter cibinongensis,146475,Acetobacter cibinongensis,Acetobacter cibinongensis
4,Acetobacter indonesiensis,104101,Acetobacter indonesiensis,Acetobacter indonesiensis
5,Acetobacter pasteurianus,438,Acetobacter pasteurianus,Acetobacter pasteurianus


In [80]:
file_tax_nodesLink = "/mnt/d/linux/W/NCBI/taxonamy20180810/nodesLink"
df_node = pd.read_csv(file_tax_nodesLink, sep = "\t",dtype = str)
print(df_node.shape)
dc_rank = dict(zip(list(df_node.loc[:,"From"]), list(df_node.loc[:,"rank"])))

(1832355, 3)


In [81]:
dc_human_bac = {e:dc_rank[e] for e in taxids}

In [82]:
Counter(dc_human_bac.values())

Counter({'species': 2661, 'subspecies': 22, 'no rank': 4})

In [83]:
print('no rank taxids:',' '.join([k for k,v in dc_human_bac.items() if v=='no rank']))

no rank taxids: 149539 54388 90370 90371


In [84]:
print('subspecies taxids:',' '.join([k for k,v in dc_human_bac.items() if v=='subspecies']))

subspecies taxids: 1690 1682 302911 264 1463164 574 39831 1585 29397 33965 319705 1962118 1770 1203599 45261 59203 1077965 1359 1458253 1077464 168 85581


### HumanRef table
Generate a table of taxid to name.  
For all HumanRef taxids, their children all match to the new name.
For example, taxid 28901 contains 2981 subspecies or strands, match all those ids to taxid 28901. Except for taxid 149539, a id in the HumanRef table, all taxids under 149539 will have the name of 149539.


In [85]:
dc_node = dict(zip(list(df_node.loc[:,"From"]), list(df_node.loc[:,"To"])))

In [86]:
file_tax_lineage = '/mnt/d/linux/W/NCBI/taxonamy20180810/speciesLineage'
dc_human_tax_children = {e:set() for e in taxids_s}
for _l in open(file_tax_lineage):
    _es = _l.split()
    for _n,_e in enumerate(_es):
        if _e in dc_human_tax_children:
            dc_human_tax_children[_e].update(_es[:_n+1])

In [87]:
print('taxid 149539 have',len(dc_human_tax_children['149539']),'and taxid 28901 have', len(dc_human_tax_children['28901']))

taxid 149539 have 568 and taxid 28901 have 2990


generate a list with `taxid_humanRef, taxid_children, number_of_children`, then keep unique `taxid_children`, keep the one with a smaller `number_of_children`

In [88]:
ls_taxid_humanRef = []
for _k,_v in dc_human_tax_children.items():
    _l = len(_v)
    for _c in _v:
        ls_taxid_humanRef.append([_k,_c,_l])

In [89]:
ls_taxid_humanRef[:6]

[['985002', '985002', 3],
 ['985002', '1418388', 3],
 ['985002', '1418406', 3],
 ['59892', '59892', 1],
 ['257274', '1210096', 2],
 ['257274', '257274', 2]]

In [90]:
df_taxid_humanRef = pd.DataFrame(ls_taxid_humanRef)

In [91]:
df_taxid_humanRef.head()

,0,1,2
0,985002,985002,3
1,985002,1418388,3
2,985002,1418406,3
3,59892,59892,1
4,257274,1210096,2


In [92]:
df_taxid_humanRef_sorted = df_taxid_humanRef.sort_values(by=[1,2])
df_taxid_humanRef_sorted.head()

,0,1,2
2272,100053,100053,9
16428,562,1000560,3286
29412,287,1000561,382
32396,216816,1000567,38
148,1328,1000570,15


In [93]:
df_taxid_humanRef_sorted[df_taxid_humanRef_sorted[1]=='149539']

,0,1,2
12490,149539,149539,568
23699,28901,149539,2990


In [94]:
df_taxid_humanRef_sorted_filter = df_taxid_humanRef_sorted.drop_duplicates(subset=[1],keep='first')
print(df_taxid_humanRef_sorted.shape, df_taxid_humanRef_sorted_filter.shape)
df_taxid_humanRef_sorted_filter[df_taxid_humanRef_sorted_filter[1]=='149539']

(32755, 3) (31770, 3)


,0,1,2
12490,149539,149539,568


In [95]:
df_taxid_humanRef_lineage = df_taxid_humanRef_sorted_filter.loc[:,[1,0]].copy()
df_taxid_humanRef_lineage.columns = ['tax_id','species_id']
df_taxid_humanRef_lineage.head()

,tax_id,species_id
2272,100053,100053
16428,1000560,562
29412,1000561,287
32396,1000567,216816
148,1000570,1328


HumanRef.speciesLineageName

In [96]:
df_speciesLineageName = pd.read_csv('/mnt/d/linux/W/NCBI/taxonamy20180810/speciesLineageName',dtype=str, index_col=0,sep='\t')
df_speciesLineageName.head()

,species_id,name,species,genus,family,order,class,phylum,kingdom
tax_id,,,,,,,,,
7,7,Azorhizobium caulinodans,Azorhizobium caulinodans,Azorhizobium,Xanthobacteraceae,Rhizobiales,Alphaproteobacteria,Proteobacteria,Bacteria
9,9,Buchnera aphidicola,Buchnera aphidicola,Buchnera,Erwiniaceae,Enterobacterales,Gammaproteobacteria,Proteobacteria,Bacteria
11,11,Cellulomonas gilvus,Cellulomonas gilvus,Cellulomonas,Cellulomonadaceae,Micrococcales,Actinobacteria,Actinobacteria,Bacteria
14,14,Dictyoglomus thermophilum,Dictyoglomus thermophilum,Dictyoglomus,Dictyoglomaceae,Dictyoglomales,Dictyoglomia,Dictyoglomi,Bacteria
17,17,Methylophilus methylotrophus,Methylophilus methylotrophus,Methylophilus,Methylophilaceae,Nitrosomonadales,Betaproteobacteria,Proteobacteria,Bacteria


In [97]:
df_speciesLineageName = df_speciesLineageName.drop(columns=['species_id'])

In [98]:
df_speciesLineageName.index = [str(e) for e in df_speciesLineageName.index]

In [99]:
df_taxid_humanRef_lineage_1 = df_taxid_humanRef_lineage.join(df_speciesLineageName,how='left',on='tax_id')

In [100]:
df_taxid_humanRef_lineage_1.head()

,tax_id,species_id,name,species,genus,family,order,class,phylum,kingdom
2272,100053,100053,Leptospira alexanderi,Leptospira alexanderi,Leptospira,Leptospiraceae,NaN,Spirochaetia,Spirochaetes,Bacteria
16428,1000560,562,Escherichia coli VDP,Escherichia coli,Escherichia,Enterobacteriaceae,Enterobacterales,Gammaproteobacteria,Proteobacteria,Bacteria
29412,1000561,287,Pseudomonas aeruginosa AES-1R,Pseudomonas aeruginosa,Pseudomonas,Pseudomonadaceae,Pseudomonadales,Gammaproteobacteria,Proteobacteria,Bacteria
32396,1000567,216816,Bifidobacterium longum 3_1_37DFAAB,Bifidobacterium longum,Bifidobacterium,Bifidobacteriaceae,Bifidobacteriales,Actinobacteria,Actinobacteria,Bacteria
148,1000570,1328,Streptococcus anginosus SK52 = DSM 20563,Streptococcus anginosus,Streptococcus,Streptococcaceae,Lactobacillales,Bacilli,Firmicutes,Bacteria


change "species" to the name in HumanRef

In [101]:
dc_speciesRefName = dict(zip(df_species['taxid'],df_species['name_use']))
print(len(dc_speciesRefName))

2687


In [102]:
df_taxid_humanRef_lineage_1['species'] = df_taxid_humanRef_lineage_1['species_id'].apply(lambda x:dc_speciesRefName[x])

In [103]:
df_taxid_humanRef_lineage_1.to_csv('/mnt/d/linux/W/NCBI/MoonRef20180810/HumanRef2018.speciesLineageName',sep='\t',index=None)